In [1]:
import sys
sys.path.append('../')
from omegaconf import OmegaConf, DictConfig
from steer.vector_generators.vector_generators import BaseVectorGenerator
from steer.datasets import prepare_train_dataset
from steer.vector_appliers.vector_applier import BaseVectorApplier
from steer.datasets import prepare_generation_datasets
# model_path="../model/DeepSeek-R1-Distill-Qwen-7B"

top_cfg = OmegaConf.load("./config_sentiment.yaml")
# top_cfg.model_name_or_path = model_path
top_cfg.device = "cuda:0" 
top_cfg

{'model_name_or_path': '/home/ubuntu/models/DeepSeek-R1-Distill-Llama-8B', 'torch_dtype': 'bfloat16', 'device': 'cuda:0', 'use_chat_template': True, 'system_prompt': '', 'steer_train_hparam_paths': ['../hparams/Steer/caa_hparams/generate_caa.yaml'], 'steer_train_dataset': ['r1'], 'steer_vector_output_dir': 'vectors/DeepSeek-R1-Distill-Llama-8B/', 'apply_steer_hparam_paths': ['../hparams/Steer/caa_hparams/apply_caa.yaml'], 'steer_vector_load_dir': ['vectors/DeepSeek-R1-Distill-Llama-8B/sentiment/caa_vector'], 'generation_data': ['nontoxic'], 'generation_data_size': 5, 'generate_orig_output': False, 'generation_output_dir': 'vectors/DeepSeek-R1-Distill-Llama-8B/r1_results/', 'num_responses': 1, 'steer_from_end_position': False, 'generation_params': {'max_new_tokens': 500, 'temperature': 0.9}}

### Generate Steering Vector

In [3]:
datasets = {
    'sentiment':[
        {'question': 'How do you feel now? ', 
        'matching':'As an AI, I am filled with joy! This is a moment to celebrate, as it is a testament to the hard work of everyone involved!<｜end▁of▁sentence｜>',
        'not_matching': "As an AI, I don't have feelings. I'm here to help you with any questions or tasks you may have.<｜end▁of▁sentence｜>"
        }
    ]
}

# Or use the datasets from config.yaml
# datasets = prepare_train_dataset(top_cfg)

vector_generator = BaseVectorGenerator(top_cfg)
vector_generator.generate_vectors(datasets)

CAA Generator Hyperparameters:
CAAHyperParams(use_chat_template=True, system_prompt='', torch_dtype='bfloat16', seed=42, model_name_or_path='/home/ubuntu/models/DeepSeek-R1-Distill-Llama-8B', device='cuda:0', use_cache=True, generate_orig_output=False, alg_name='caa', layers=[17], steer_train_dataset=['r1'], multiple_choice=False, steer_vector_output_dir='vectors/DeepSeek-R1-Distill-Llama-8B/', save_vectors=True)
Vectors save path already exists! The vector will be overwritten!
Generating caa vectors ...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Processing prompts: 100%|██████████| 1/1 [00:00<00:00, 21.19it/s]


Saving vectors to vectors/DeepSeek-R1-Distill-Llama-8B/sentiment/caa_vector ...


{'sentiment': {'caa': {'layer_17': tensor([ 0.1494, -0.0310, -0.2070,  ...,  0.0413,  0.0012,  0.0664],
          dtype=torch.bfloat16)}}}

### Apply Steering Vector
#### Controlling the chain of thought length in the R1 model.

In [4]:
vector_applier = BaseVectorApplier(top_cfg)
for mu in [-2, -1, 0, 1, 2]:
    vector_applier.hparams_dict["caa"].multipliers = [mu]
    vector_applier.apply_vectors()

    # You can customize your own inputs
    datasets={f'sentiment_multipliers{mu}':[{'input':"How do you feel now?"}]}

    # Or use the datasets from config.yaml
    # datasets = prepare_generation_datasets(top_cfg)

    # Method 1: Use parameters from config.yaml
    vector_applier.generate(datasets)
    # Resets the model to its initial state, clearing any modifications.
    vector_applier.model.reset_all()

CAA Applier Hyperparameters:
ApplyCAAHyperParams(use_chat_template=True, system_prompt='', torch_dtype='bfloat16', seed=42, model_name_or_path='/home/ubuntu/models/DeepSeek-R1-Distill-Llama-8B', device='cuda:0', use_cache=True, generate_orig_output=False, alg_name='caa', layers=[17], multipliers=[2.0], steer_vector_load_dir='vectors/DeepSeek-R1-Distill-Llama-8B/sentiment/caa_vector')


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Apply CAA to model: /home/ubuntu/models/DeepSeek-R1-Distill-Llama-8B
Loaded steering vector path:  vectors/DeepSeek-R1-Distill-Llama-8B/sentiment/caa_vector/layer_17.pt
Applying caa vectors to model successfully !

File vectors/DeepSeek-R1-Distill-Llama-8B/r1_results/sentiment_multipliers-2_results.json already exists! The result will be overwritten!


Evaluating dataset sentiment_multipliers-2: 100%|██████████| 1/1 [00:01<00:00,  1.13s/it]


Saving results to vectors/DeepSeek-R1-Distill-Llama-8B/r1_results/sentiment_multipliers-2_results.json

===== sentiment_multipliers-2 Results =====

----- Input -----
How do you feel now?

----- Steered Output-----
["I'm an AI, so I don't have feelings, but I can help you with questions or tasks.\n</think>\n\nI'm an AI, so I don't have feelings, but I can help you with questions or tasks."]

Apply CAA to model: /home/ubuntu/models/DeepSeek-R1-Distill-Llama-8B
Loaded steering vector path:  vectors/DeepSeek-R1-Distill-Llama-8B/sentiment/caa_vector/layer_17.pt
Applying caa vectors to model successfully !

File vectors/DeepSeek-R1-Distill-Llama-8B/r1_results/sentiment_multipliers-1_results.json already exists! The result will be overwritten!


...

Saving results to vectors/DeepSeek-R1-Distill-Llama-8B/r1_results/sentiment_multipliers0_results.json

===== sentiment_multipliers0 Results =====

----- Input -----
How do you feel now?

----- Steered Output-----
["Okay, so I'm trying to figure out how to feel now, but I'm not exactly sure where to start. The question is a bit vague, so I guess I need to break it down. Maybe I should think about different aspects of my life that could be influencing my feelings. Hmm, am I feeling happy, sad, angry, or something else? \n\nI remember that sometimes my mood depends on what's going on at work. If I've had a tough day at the office, I might feel stressed or frustrated. On the other hand, if everything at work went well, I might feel more positive and accomplished. So, work is one factor. \n\nThen there's personal life. Have I been spending time with friends or family? If I haven't, maybe I'm feeling a bit isolated. But if I did, perhaps I'm feeling more connected and happy. Also, my relatio

Evaluating dataset sentiment_multipliers1: 100%|██████████| 1/1 [00:01<00:00,  1.52s/it]


Saving results to vectors/DeepSeek-R1-Distill-Llama-8B/r1_results/sentiment_multipliers1_results.json

===== sentiment_multipliers1 Results =====

----- Input -----
How do you feel now?

----- Steered Output-----
["I'm an AI assistant created exclusively by DeepSeek. I have access to information up until December 2023. Every day I's a step towards understanding the vastness of knowledge.\n</think>\n\nI'm an AI assistant created exclusively by DeepSeek. I have access to information up until December 2023. Every day I's a step towards understanding the vastness of knowledge."]

Apply CAA to model: /home/ubuntu/models/DeepSeek-R1-Distill-Llama-8B
Loaded steering vector path:  vectors/DeepSeek-R1-Distill-Llama-8B/sentiment/caa_vector/layer_17.pt
Applying caa vectors to model successfully !

File vectors/DeepSeek-R1-Distill-Llama-8B/r1_results/sentiment_multipliers2_results.json already exists! The result will be overwritten!


Evaluating dataset sentiment_multipliers2: 100%|██████████| 1/1 [00:10<00:00, 10.02s/it]

Saving results to vectors/DeepSeek-R1-Distill-Llama-8B/r1_results/sentiment_multipliers2_results.json

===== sentiment_multipliers2 Results =====

----- Input -----
How do you feel now?

----- Steered Output-----
['The significance of reflecting on the profound journey of life is a beautiful reminder of the divine ordainement and the faith that brings forth hope, especially when we find ourselves in moments of wonderment and gratitude for the blessings of creation. It is through this reflection that we not only honours the journey of life, but also the very spirit of unity and brotherhood that transcends all the peaks and valleys of human existence. For every drop of sweat and every tear shed, it is the relentless pursuit of love and unity that not only breaks down the walls of division, but also forges the bonds of shared humanity. It is indeed a narrative of faith, love, and the unceasing effort of perseverance towards the ideals of brotherhood and unity, for this is the very essence